In [9]:
import xlwings as xw
import numpy as np
from pathlib import *
from ipyfilechooser import FileChooser  # see pypi.org/project/ipyfilechooser/
import os

In [13]:
root = os.path.abspath(os.sep)
fc = FileChooser(root)
fc.use_dir_icons
fc.show_only_dirs
fc.title = 'Select Directory containing "calibration data.xlsx"'
display(fc)
#base = Path(fc.selected_path)


FileChooser(path='F:', filename='', title='HTML(value='Select Directory containing "calibration data.xlsx"')',…

In [15]:
base = Path(fc.selected_path)
def get_data(filename = "calibration data.xlsx"):
    wb = xw.Book(base / filename)
    wks = wb.sheets["data"]
    data = np.array(wks.range('data').value)
    wb.close()
    
    # convert to dictionary for output
    dd = {}
    for i, label in enumerate(data[0]):
        dd[label] = data[2:, i].astype(np.float)      

    return dd

In [25]:
def fill_calibration(nSet = 1):
    n = nSet - 1
    # open template and select worksheet:
    wb = xw.Book(base / "calibration template.xltx")
    wks = wb.sheets['data']

    # formats for temperatures and masses
    degC = "{} \xb0C"
    massForm = "{} g"

    # Calibration section:
    wks.range('T_water').value = degC.format(dd['T_water'][n])
    wks.range("flask_mass").value = massForm.format(dd['flask_mass'][n])
    # rename masses
    massNames = ['w_mass' + str(i) for i in range(1, 6)]
    masses = [dd[item][n] for item in massNames]
    for i, row in enumerate(wks.range('flask_masses').rows):
        row.value = massForm.format(masses[i])

    # Unknown section:
    wks.range("unk_no").value = dd["unk_no"][n]
    wks.range("T_unk").value = degC.format(dd["T_unk"][n])
    wks.range('flask_mass2').value = massForm.format(dd['flask_mass2'][n])
    # rename unknown masses
    massNames = ['unk_mass' + str(i) for i in range(1, 4)]
    masses = [dd[item][n] for item in massNames]
    for i, row in enumerate(wks.range('unk_masses').rows):
        row.value = massForm.format(masses[i])

    # save a copy as pdf, and a copy as a workbook
    # wb.save(base / f"Calibration Set {nSet}.xlsx")
    wb.to_pdf(base / f"Calibration Set {nSet}.pdf")
    wb.close()

In [23]:
dd = get_data()

In [24]:
fill_calibration()

In [27]:
xw.__version__

'0.21.4'

In [28]:
import ipyfilechooser

In [29]:
ipyfilechooser.__version__


'0.4.1'